# Homework 1, Scalable Data
As part of the data team, you need to transform various datasets to create the following:

1. The table of extended quantites per component for all programs.
2. The table of total cost per program per vendor, based on the latest received quote.
3. The table of total cost per program per vendor, based on the first received quote.
4. The table of "best-in-class" total cost per program (regardless of vendor).

These four tables must be the final outputs of your notebook. If not, we will not grade your assignment beyond the attempt. Our TAs will not execute your code. The notebook you upload to Canvas must have been executed, with all tables shown as output.

In [16]:
import polars as pl
from datetime import datetime

## Part 1: Extended quantities per component for all programs
Task: 
A table detailing the total server rack quantity for each component across all programs. Each record in the table should represent a distinct component. Each attribute of the table should represent a distinct program. The intersection of record and attribute should represent the total extended quantity of parts for that component and program combination.

Process:
First, we want to get all of the scaled quantities in one table. I multiplied the amount of servers listed for each program to every item in `server_specs`. Then, I concatenated the remaining of the items for `rack_specs` to get a list of scaled quantities to allow us to calculate the total cost per program per vendor.

In [13]:
# read in csv files and their respective sheets
server_specs = pl.read_excel('program configurations.xlsx', sheet_name='server_specs')
rack_specs = pl.read_excel('program configurations.xlsx', sheet_name='rack_specs')


In [ ]:
# filter rack_specs df to get only row where itme is SERVERS
# remove item column to only leave numerical quantities for each server
# then, get only first row from filtered df to get the quantities which outputs [12,8,14,10,10]
servers_quantities = rack_specs.filter(pl.col('Item') == 'SERVERS').select(pl.exclude('Item')).row(0)

# multiply each component in server_specs by the corresponding SERVERS quantity
# we want (pl.col('Server A') * servers_quantities[0]).alias('Server A') for each server

server_columns = ['Server A', 'Server B', 'Server C', 'Server D', 'Server E']
scaled_servers_quantities = server_specs.with_columns([
    (pl.col(server) * servers_quantities[i]).alias(server) 
    for i, server in enumerate(server_columns)
])

In [14]:
rack_quantities = rack_specs.filter(pl.col('Item').is_in(['TOR', 'CHASSIS']))

extended_quantities_df = pl.concat([scaled_servers_quantities, rack_quantities], how='vertical')

print("Table with Extended Quantities Per Component for All Programs")
extended_quantities_df

Table with Extended Quantities Per Component for All Programs


Item,Server A,Server B,Server C,Server D,Server E
str,i64,i64,i64,i64,i64
"""CPU""",24,16,28,10,10
"""GPU""",0,32,0,0,20
"""RAM""",48,32,56,80,80
"""SSD""",12,16,14,10,0
"""HDD""",0,0,0,200,200
…,…,…,…,…,…
"""NIC""",24,16,28,20,10
"""PSU""",12,16,14,10,10
"""TRAY""",12,8,14,10,10


## Part 2: Data Validation Requirements
We need to calculate the total cost per program/server per vendor based on the latest received quote. Therefore, we'll first need to filter the `quote_lines.csv` by our data validation requirements given at the beginning which were:
- Quotes for a given month can only be accepted starting on the first Monday of the month and ending on the 25th. These boundary dates are inclusive. Any quotes provided outside of these dates will not be considered.
- The sum of the quote line details must match the provided quote summary total. If the two datasets do not agree, the quotes will not be considered.
- We do not purchase Program C and Program E quotes from Vendor 7. Vendor 7’s systems are configured to quote all available programs, so these quotes need to be discarded.



### Date Boundaries - First Monday of Month - 25th  
Filtering the dates first for both `quote_lines.csv` and `quote_summaries.csv` resulting in:

- `filtered_quote_lines`
- `filtered_quote_summaries_df` 

In [77]:
# First we want to filter out the quotes for the first Monday - 25th
# I am printing the og csv file so you can see the that all days of Sept are included
quote_lines_df = pl.read_csv('quote_lines.csv')

# Convert 'quote_timestamp' to datetime
quote_lines_df = quote_lines_df.with_columns([
    pl.col("quote_timestamp").str.to_datetime().alias("quote_timestamp")
])

quote_lines_df.shape


(175, 14)

In [137]:
# We only have September 1, 2024 - September 30, 2024 in our data
# I have made it sort of dynamic if we were to add more months

def get_first_monday(d: datetime.date) -> datetime.date:
    first_day = d.replace(day=1)
    days_to_monday = (7 - first_day.weekday()) % 7
    return first_day + datetime.timedelta(days=days_to_monday)

# Dynamically calculate the first Monday for reference date which I set as September 2024
reference_date = datetime.date(2024, 9, 1)
first_monday = get_first_monday(reference_date)

# Set the end date dynamically as the 25th of the same month
end_date = reference_date.replace(day=26)

# Filter the DataFrame based on the dynamic date range (first Monday to the 25th of the month)
filtered_quote_lines = quote_lines_df.filter(
    (pl.col("quote_timestamp") >= pl.lit(first_monday)) &
    (pl.col("quote_timestamp") <= pl.lit(end_date))
)
filtered_quote_lines


Vendor,Program,quote_timestamp,CPU,GPU,RAM,SSD,HDD,MOBO,NIC,PSU,TRAY,TOR,CHASSIS
str,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Vendor_7""","""Program_D""",2024-09-25 03:52:19.637095,305.84,409.06,53.43,196.14,103.49,107.72,20.98,94.19,11.57,746.67,1015.84
"""Vendor_6""","""Program_E""",2024-09-12 00:26:22.414219,324.21,446.35,44.91,194.27,184.53,100.12,28.8,89.99,70.05,728.08,1068.57
"""Vendor_5""","""Program_A""",2024-09-23 18:35:08.078890,300.29,414.38,48.73,187.55,105.42,103.5,21.64,102.29,22.98,734.33,1770.98
"""Vendor_5""","""Program_B""",2024-09-24 11:33:41.034306,333.53,445.41,48.16,193.25,104.02,90.87,23.0,113.9,80.13,616.13,1563.11
"""Vendor_7""","""Program_C""",2024-09-15 13:20:11.557807,324.79,458.29,48.68,180.91,193.3,114.81,25.95,86.8,76.02,629.8,1437.87
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Vendor_7""","""Program_E""",2024-09-02 19:59:44.793380,318.99,447.29,48.28,190.64,152.61,80.35,23.62,98.91,78.02,719.53,1614.46
"""Vendor_7""","""Program_C""",2024-09-10 20:59:39.959689,345.09,477.63,49.35,189.19,124.61,86.75,28.77,112.67,23.06,727.32,1646.63
"""Vendor_4""","""Program_E""",2024-09-11 17:20:46.390587,313.77,476.99,46.76,191.12,122.7,81.18,28.76,115.58,27.46,711.28,1094.2


In [ ]:
filtered_quote_lines = filtered_quote_lines.filter(
    ~((pl.col("Vendor") == "Vendor_7") & (pl.col("Program").is_in(["Program_C", "Program_E"])))
)

# Output the results
print("The first monday on the month is:", first_monday)
print("Filtered Quote Lines based on Dates and Program Exclusion:")
filtered_quote_lines

The first monday on the month is: 2024-09-02
Filtered Quote Lines based on Dates and Program Exclusion:


Vendor,Program,quote_timestamp,CPU,GPU,RAM,SSD,HDD,MOBO,NIC,PSU,TRAY,TOR,CHASSIS
str,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Vendor_7""","""Program_D""",2024-09-25 03:52:19.637095,305.84,409.06,53.43,196.14,103.49,107.72,20.98,94.19,11.57,746.67,1015.84
"""Vendor_6""","""Program_E""",2024-09-12 00:26:22.414219,324.21,446.35,44.91,194.27,184.53,100.12,28.8,89.99,70.05,728.08,1068.57
"""Vendor_5""","""Program_A""",2024-09-23 18:35:08.078890,300.29,414.38,48.73,187.55,105.42,103.5,21.64,102.29,22.98,734.33,1770.98
"""Vendor_5""","""Program_B""",2024-09-24 11:33:41.034306,333.53,445.41,48.16,193.25,104.02,90.87,23.0,113.9,80.13,616.13,1563.11
"""Vendor_3""","""Program_A""",2024-09-17 18:56:17.120014,338.88,441.63,51.69,198.78,110.31,113.62,23.77,112.67,32.99,547.85,1108.16
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Vendor_5""","""Program_A""",2024-09-05 03:11:32.722222,330.5,468.04,47.79,190.65,110.16,91.25,25.52,94.26,38.55,684.71,1268.8
"""Vendor_4""","""Program_C""",2024-09-08 16:45:01.071635,338.68,466.55,50.5,190.87,142.77,102.34,29.21,116.78,93.16,617.82,1820.72
"""Vendor_4""","""Program_E""",2024-09-11 17:20:46.390587,313.77,476.99,46.76,191.12,122.7,81.18,28.76,115.58,27.46,711.28,1094.2


### Calculuate Server Rack Quantites 

In [140]:
# Create a mapping between Program and Server columns
program_to_server = {
    "Program_A": "Server A",
    "Program_B": "Server B",
    "Program_C": "Server C",
    "Program_D": "Server D",
    "Program_E": "Server E"
}

# List of component columns to scale (CPU - CHASSIS)
components = ["CPU", "GPU", "RAM", "SSD", "HDD", "MOBO", "NIC", "PSU", "TRAY", "TOR", "CHASSIS"]

# Function to scale components by quantities based on the program
def scale_components(df, extended_quantities_df):
    # For each program, get the corresponding server and scale the components
    for program, server in program_to_server.items():
        # Get the rows corresponding to the program
        program_mask = df["Program"] == program
        
        # Scale each component based on the quantities in the extended_quantities_df
        for component in components:
            quantity_col = extended_quantities_df.filter(pl.col("Item") == component)[server].item()
            df = df.with_columns([
                pl.when(program_mask).then(pl.col(component) * quantity_col).otherwise(pl.col(component)).alias(component)
            ])
    
    return df

# Scale the filtered_quote_lines components by the quantities from the extended_quantities_df
scaled_quote_lines = scale_components(filtered_quote_lines, extended_quantities_df)

# Create a new column 'total_cost' by summing all component costs for each row
scaled_quote_lines = scaled_quote_lines.with_columns([
    sum([pl.col(component) for component in components]).alias("total_cost")
])

# Output the final DataFrame with the total cost
print(scaled_quote_lines)


shape: (155, 15)
┌──────────┬───────────┬──────────────────┬─────────┬───┬─────────┬─────────┬─────────┬────────────┐
│ Vendor   ┆ Program   ┆ quote_timestamp  ┆ CPU     ┆ … ┆ TRAY    ┆ TOR     ┆ CHASSIS ┆ total_cost │
│ ---      ┆ ---       ┆ ---              ┆ ---     ┆   ┆ ---     ┆ ---     ┆ ---     ┆ ---        │
│ str      ┆ str       ┆ datetime[μs]     ┆ f64     ┆   ┆ f64     ┆ f64     ┆ f64     ┆ f64        │
╞══════════╪═══════════╪══════════════════╪═════════╪═══╪═════════╪═════════╪═════════╪════════════╡
│ Vendor_7 ┆ Program_D ┆ 2024-09-25       ┆ 3058.4  ┆ … ┆ 115.7   ┆ 746.67  ┆ 1015.84 ┆ 34309.11   │
│          ┆           ┆ 03:52:19.637095  ┆         ┆   ┆         ┆         ┆         ┆            │
│ Vendor_6 ┆ Program_E ┆ 2024-09-12       ┆ 3242.1  ┆ … ┆ 700.5   ┆ 728.08  ┆ 1068.57 ┆ 57354.15   │
│          ┆           ┆ 00:26:22.414219  ┆         ┆   ┆         ┆         ┆         ┆            │
│ Vendor_5 ┆ Program_A ┆ 2024-09-23       ┆ 7206.96 ┆ … ┆ 275.76  ┆ 734.33

### Filter Out Program C & E From Vendor 7

In [ ]:
# Also dropping the out of bound dates for quote_summaries.csv

quote_summaries_df = pl.read_csv('quote_summaries.csv')

# Convert 'quote_timestamp' to datetime
quote_summaries_df = quote_summaries_df.with_columns([
    pl.col("quote_timestamp").str.to_datetime().alias("quote_timestamp")
])

filtered_quote_summaries_df = quote_summaries_df.filter(
    (pl.col("quote_timestamp") >= pl.lit(first_monday)) &
    (pl.col("quote_timestamp") <= pl.lit(end_date))
)

filtered_quote_summaries_df = filtered_quote_summaries_df.filter(
    ~((pl.col("vendor") == "Vendor_7") & (pl.col("program").is_in(["C", "E"])))
)

filtered_quote_summaries_df

### Comparing Outputs to Ensure Quote Line and Quote Summary totals match
Merged `scaled_quote_lines` and `filtered_quote_summaries_df` and compared the summary total and itemized total

In [141]:
# Define the mapping function to convert single letters to "Program_" format
def relabel_program(program):
    program_mapping = {
        "A": "Program_A",
        "B": "Program_B",
        "C": "Program_C",
        "D": "Program_D",
        "E": "Program_E"
    }
    return program_mapping.get(program, program)  # If not found, return the original value

# Apply the relabeling to the 'program' column using map_elements with return_dtype specified
filtered_quote_summaries_df = filtered_quote_summaries_df.with_columns(
    pl.col("program").map_elements(relabel_program, return_dtype=pl.Utf8).alias("program")
)

# Now perform the join operation between filtered_quote_summaries_df and scaled_quote_lines
merged_df = filtered_quote_summaries_df.join(
    scaled_quote_lines.select(["Vendor", "Program", "quote_timestamp", "total_cost"]),
    left_on=["vendor", "program", "quote_timestamp"],  # 'program' in filtered_quote_summaries_df
    right_on=["Vendor", "Program", "quote_timestamp"],  # 'Program' in scaled_quote_lines
    how="inner"  # Perform an inner join
)

merged_df.shape

(155, 5)

### Printing Mismatched data

In [142]:
# Define a small epsilon value for floating-point comparison tolerance
epsilon = 1e-6  # You can adjust this value as needed based on the precision in your data

# Step 1: Compare 'reported_total_price' with 'total_cost' using a tolerance
mismatches_df = merged_df.filter(
    (pl.col("reported_total_price") - pl.col("total_cost")).abs() > epsilon
)

# Step 2: Output mismatches if they exist
if mismatches_df.height > 0:
    print("\nDiscrepancies found between reported_total_price and total_cost:")
    print(mismatches_df)
else:
    print("\nAll values match between reported_total_price and total_cost within the tolerance.")



Discrepancies found between reported_total_price and total_cost:
shape: (12, 5)
┌──────────┬───────────┬────────────────────────────┬──────────────────────┬────────────┐
│ vendor   ┆ program   ┆ quote_timestamp            ┆ reported_total_price ┆ total_cost │
│ ---      ┆ ---       ┆ ---                        ┆ ---                  ┆ ---        │
│ str      ┆ str       ┆ datetime[μs]               ┆ f64                  ┆ f64        │
╞══════════╪═══════════╪════════════════════════════╪══════════════════════╪════════════╡
│ Vendor_5 ┆ Program_B ┆ 2024-09-24 11:33:41.034306 ┆ 31493.78             ┆ 30576.49   │
│ Vendor_1 ┆ Program_A ┆ 2024-09-20 17:15:49.069230 ┆ 19332.6              ┆ 18769.51   │
│ Vendor_6 ┆ Program_E ┆ 2024-09-05 02:24:06.963283 ┆ 54307.11             ┆ 52725.35   │
│ Vendor_1 ┆ Program_E ┆ 2024-09-24 10:23:03.302944 ┆ 54118.35             ┆ 52542.09   │
│ Vendor_5 ┆ Program_A ┆ 2024-09-10 15:42:36.421205 ┆ 17677.62             ┆ 17162.74   │
│ …        ┆ …     

### Merged Dataframe That Complies with Data Validation Requirements  

In [143]:
validated_merged = merged_df.filter(
    (pl.col("reported_total_price") - pl.col("total_cost")).abs() <= epsilon
)
print("\nValidated Merged DataFrame (with matching total costs):")
validated_merged


Validated Merged DataFrame (with matching total costs):


vendor,program,quote_timestamp,reported_total_price,total_cost
str,str,datetime[μs],f64,f64
"""Vendor_7""","""Program_D""",2024-09-25 03:52:19.637095,34309.11,34309.11
"""Vendor_6""","""Program_E""",2024-09-12 00:26:22.414219,57354.15,57354.15
"""Vendor_5""","""Program_A""",2024-09-23 18:35:08.078890,17566.51,17566.51
"""Vendor_3""","""Program_A""",2024-09-17 18:56:17.120014,18337.45,18337.45
"""Vendor_6""","""Program_B""",2024-09-03 02:21:46.569968,32021.13,32021.13
…,…,…,…,…
"""Vendor_5""","""Program_A""",2024-09-05 03:11:32.722222,17768.43,17768.43
"""Vendor_4""","""Program_C""",2024-09-08 16:45:01.071635,22611.56,22611.56
"""Vendor_4""","""Program_E""",2024-09-11 17:20:46.390587,45293.58,45293.58


# Part 3: Calculate Total Cost Per Program Per Vendor